<a href="https://colab.research.google.com/github/seopp/Tobigs19/blob/main/Week7/week7_CNNbasic_AlexNet_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNNbasic Assignment#2

# AlexNet 구현

모델 구현 후 summary로 전체 모델 구조 출력과 주석으로 간단한 설명을 달아주시면 됩니다.

프레임워크는 자유이고, 기본 tensforflow와 pytorch tutorial 사이트를 아래에 첨부해 드립니다.

이 외 각 프레임워크 별 summary 등 추가적인 사용 방법은 구글링으로 찾아주세요!-!

- Tensorflow Tutorial: https://www.tensorflow.org/tutorials?hl=ko

- Pytorch Tutorial: https://tutorials.pytorch.kr/

![image-2.png](attachment:image-2.png)

## Pytorch

In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
print(torch.__version__)
print(torchvision.__version__)

1.13.1+cu116
0.14.1+cu116


In [2]:
feature_ = [
            [3, 96, 11, 4, 0, "pool"],
            [96, 256, 5, 1, 2, "pool"],
            [256, 384, 3, 1, 1, "no"],
            [384, 384, 3, 1, 1, "no"],
            [384, 256, 3, 1, 1, "pool"],
            ]


classifier_ = [
               [9216, 4096],
               [4096, 4096],
               [4096]
               ]

In [3]:
class AlexNet(nn.Module):

    def __init__(self, feature_, classifier_, n_classes):
        super().__init__()
        
        self.feature_ = feature_
        self.classifier_ = classifier_
        self.n_classes = n_classes

        self.feature = self.make_feature()
        self.flatten = nn.Flatten()
        self.classifier = self.make_classifier()


    def make_feature(self):
        layers = []
        for i in self.feature_:
            layers.append(nn.Conv2d(i[0], i[1], i[2], i[3], i[4]))
            layers.append(nn.ReLU())
            if i[5] == "pool": layers.append(nn.MaxPool2d(3, 2))
            layers.append(nn.BatchNorm2d(i[1]))
        return nn.Sequential( *layers )


    def make_classifier(self):
        layers = []
        for i in self.classifier_:
            if i == self.classifier_[-1]: i = [i[0], self.n_classes] 
            layers.append(nn.Linear(i[0], i[1]))
            layers.append(nn.ReLU())
        return nn.Sequential( *layers )

    def forward(self, x):
        x = self.feature(x)
        x = self.flatten(x)
        x = self.classifier(x)
        return x

In [4]:
model = AlexNet(feature_, classifier_, 1000)

In [5]:
model

AlexNet(
  (feature): Sequential(
    (0): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (5): ReLU()
    (6): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU()
    (10): BatchNorm2d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU()
    (13): BatchNorm2d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   

In [6]:
model.feature

Sequential(
  (0): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4))
  (1): ReLU()
  (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (4): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (5): ReLU()
  (6): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (7): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (8): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (9): ReLU()
  (10): BatchNorm2d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (11): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (12): ReLU()
  (13): BatchNorm2d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (14): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU()
  (16): MaxPool2d(kernel_size=3, stride=

In [7]:
summary((model.to("cuda:0")), (3, 227, 227)) # summary code 추가

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 55, 55]          34,944
              ReLU-2           [-1, 96, 55, 55]               0
         MaxPool2d-3           [-1, 96, 27, 27]               0
       BatchNorm2d-4           [-1, 96, 27, 27]             192
            Conv2d-5          [-1, 256, 27, 27]         614,656
              ReLU-6          [-1, 256, 27, 27]               0
         MaxPool2d-7          [-1, 256, 13, 13]               0
       BatchNorm2d-8          [-1, 256, 13, 13]             512
            Conv2d-9          [-1, 384, 13, 13]         885,120
             ReLU-10          [-1, 384, 13, 13]               0
      BatchNorm2d-11          [-1, 384, 13, 13]             768
           Conv2d-12          [-1, 384, 13, 13]       1,327,488
             ReLU-13          [-1, 384, 13, 13]               0
      BatchNorm2d-14          [-1, 384,

In [10]:
# !pip install pytorch_model_summary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
# !pip install torchinfo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
import pytorch_model_summary
from torchinfo import summary

x = torch.empty((100,3, 227,227)).to("cuda:0")

print(pytorch_model_summary.summary(model, x, show_input=True))
print('!@#'*40)
summary(model, input_size = x.shape )

--------------------------------------------------------------------------
      Layer (type)            Input Shape         Param #     Tr. Param #
          Conv2d-1     [100, 3, 227, 227]          34,944          34,944
            ReLU-2      [100, 96, 55, 55]               0               0
       MaxPool2d-3      [100, 96, 55, 55]               0               0
     BatchNorm2d-4      [100, 96, 27, 27]             192             192
          Conv2d-5      [100, 96, 27, 27]         614,656         614,656
            ReLU-6     [100, 256, 27, 27]               0               0
       MaxPool2d-7     [100, 256, 27, 27]               0               0
     BatchNorm2d-8     [100, 256, 13, 13]             512             512
          Conv2d-9     [100, 256, 13, 13]         885,120         885,120
           ReLU-10     [100, 384, 13, 13]               0               0
    BatchNorm2d-11     [100, 384, 13, 13]             768             768
         Conv2d-12     [100, 384, 13,

Layer (type:depth-idx)                   Output Shape              Param #
AlexNet                                  [100, 1000]               --
├─Sequential: 1-1                        [100, 256, 6, 6]          --
│    └─Conv2d: 2-1                       [100, 96, 55, 55]         34,944
│    └─ReLU: 2-2                         [100, 96, 55, 55]         --
│    └─MaxPool2d: 2-3                    [100, 96, 27, 27]         --
│    └─BatchNorm2d: 2-4                  [100, 96, 27, 27]         192
│    └─Conv2d: 2-5                       [100, 256, 27, 27]        614,656
│    └─ReLU: 2-6                         [100, 256, 27, 27]        --
│    └─MaxPool2d: 2-7                    [100, 256, 13, 13]        --
│    └─BatchNorm2d: 2-8                  [100, 256, 13, 13]        512
│    └─Conv2d: 2-9                       [100, 384, 13, 13]        885,120
│    └─ReLU: 2-10                        [100, 384, 13, 13]        --
│    └─BatchNorm2d: 2-11                 [100, 384, 13, 13]        76

위의 방법으로 paramter와 model의 연산량을 간단하게 볼 수 있다.